In [ ]:
#Import Dataset
#==============================================
import pandas as pd
URL = "../data/processed/healthcare-dataset-stroke-data-T.csv"
df = pd.read_csv(URL)
df.head()

In [ ]:
# Split the data into features (X) and labels (y)
X = df.drop('stroke', axis=1)
y = df['stroke']

In [ ]:
!pip install xgboost

In [23]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
import optuna

In [24]:
# Define the StratifiedKFold splitting strategy
splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [28]:
def objective(trial):
    model = xgb.XGBClassifier(
            n_jobs=-1,
            max_depth=int(trial.suggest_int("max_depth", 2, 10)),
            subsample=trial.suggest_uniform("subsample", 0.1, 1),
            learning_rate=trial.suggest_float("learning_rate", 0.05, 0.3),
            n_estimators=int(trial.suggest_int("n_estimators", 100, 500)),
            gamma=trial.suggest_uniform("gamma", 0, 1),
            min_child_weight=int(trial.suggest_int("min_child_weight", 1, 10)),
        )
    
    # Get the cross-validation scores
    scores = cross_val_score(xgb_clf, X, y, cv=splitter, n_jobs=-1)
    
    # Return the mean of the scores
    return np.mean(scores)

In [29]:
# Define the study
study = optuna.create_study()

# Optimize the XGBoost classifier
study.optimize(objective, n_trials=100)

# Get the best trial
best_trial = study.best_trial

# Print the best hyperparameters
print("Best hyperparameters: ", best_trial.params)

# Train the XGBoost classifier with the best hyperparameters
xgb_clf = XGBClassifier(n_estimators=best_trial.params["n_estimators"], 
                        max_depth=best_trial.params["max_depth"], 
                        learning_rate=best_trial.params["learning_rate"], 
                        subsample=best_trial.params["subsample"], 
                        gamma=best_trial.params["gamma"], 
                        colsample_bytree=best_trial.params["colsample_bytree"])

xgb_clf.fit(X, y)

# Print the accuracy score
print("Accuracy score: ", xgb_clf.score(X, y))

[I 2023-02-11 11:58:58,885] A new study created in memory with name: no-name-1f395d8b-f74b-4220-b0e1-c6e37bdc4d4e
/tmp/ipykernel_11583/2478485479.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  subsample=trial.suggest_uniform("subsample", 0.1, 1),
/tmp/ipykernel_11583/2478485479.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  gamma=trial.suggest_uniform("gamma", 0, 1),
[I 2023-02-11 11:58:59,049] Trial 0 finished with value: 0.9997963340122199 and parameters: {'max_depth': 9, 'subsample': 0.9824708341391315, 'learning_rate': 0.056911114031618346, 'n_estimators': 260, 'gamma': 0.29857196004584907, 'min_child_weight': 8}. Best is trial 0 with v

Best hyperparameters:  {'max_depth': 9, 'subsample': 0.9824708341391315, 'learning_rate': 0.056911114031618346, 'n_estimators': 260, 'gamma': 0.29857196004584907, 'min_child_weight': 8}


KeyError: 'colsample_bytree'

In [13]:
# optimize the hyperparameters
study = optuna.create_study()
study.optimize(objective, n_trials=100)

[I 2023-02-11 11:39:26,796] A new study created in memory with name: no-name-00753156-0bb9-43a8-b827-7d4c70510dd5
/tmp/ipykernel_11583/4068781647.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "learning_rate": trial.suggest_uniform("learning_rate", 0.01, 0.2),
/tmp/ipykernel_11583/4068781647.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "gamma": trial.suggest_uniform("gamma", 0, 1),
[I 2023-02-11 11:39:26,929] Trial 0 finished with value: 0.0024691358024691024 and parameters: {'max_depth': 1, 'n_estimators': 16, 'learning_rate': 0.14441559990327923, 'gamma': 0.5734420298992503, 'min_child_weight': 8}. Best is trial 0 with value: 0.00246913

In [14]:
# print the best hyperparameters
best_params = study.best_params
print("Best params: ", best_params)

Best params:  {'max_depth': 1, 'n_estimators': 16, 'learning_rate': 0.14441559990327923, 'gamma': 0.5734420298992503, 'min_child_weight': 8}


In [16]:
# train the final model using the best hyperparameters
model = xgb.XGBClassifier(**best_params)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)